In [312]:
import pandas as pd
import numpy as np
import panel as pn
import os
import re
import yaml
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot

In [313]:
def configReader():
    """
    explanation: This function open config,yaml file 
    and fetch the gonfigue file information
    input: ...
    output: configue file
    """
    with open("config_flanker.yaml", "r") as inputFile:
        config = yaml.safe_load(inputFile)
    return config

In [314]:

def render_plot(x_axis='index', y_axis='reaction_time', plot_type='boxPlot'):
    """
    explanation: This function opens all files finds those that have data
                 and erase rest of them. thenadd some property to data and 
                 use the plot function to sketch the data.
    input: type of graph, x_axis, y_axis
    output: a set of graphs
    """
    #read file names and enter the heet names
    config = configReader()
    sheet_name = ['dehydration_1', 'dehydration_2', 'dehydration_3', 'control_1', 'control_2', 'control_3']
    graph = []

    #save the name of the sheets as the keys and data frames as values, determining color
    for flanker_test in config.values():
        flanker_test_dic = pd.read_excel(flanker_test,sheet_name=sheet_name)
        filename = os.path.basename(flanker_test)
        color = re.split("[_.]", filename)[-2]

        #erase empty cells
        for key in list(flanker_test_dic.keys()):
            if flanker_test_dic[key].empty: 
                del flanker_test_dic[key]

        #add some properties
        for key in flanker_test_dic:
            flanker_test_dic[key].columns = ['flanker', 'homology', 'accuracy', 'reaction_time']
            flanker_test_dic[key].insert(0, 'participant', color)
            flanker_test_dic[key].insert(0, 'index', flanker_test_dic[key].index)

        #making the plots
        fig = making_plots(flanker_test_dic, color, graph, x_axis, y_axis, plot_type)
        graph.append(fig)
    return(gridplot(graph, ncols=2, plot_width=450, plot_height=300))



In [315]:
def making_plots(flanker_test_dic, color, graph,x_axis, y_axis, plot_type):
    """
    explanation: This function makes plots based on plot type, x and y axises
    input: test dictionary, color, main graph, x axis, y axis, plot type
    output: figure of each color
    """
    fig = figure(title=f'Flanker Test {color}', plot_width=800, plot_height=900)
    colors = ['red', 'blue','green', 'purple', 'black']

    for counter, key in enumerate(flanker_test_dic):

            #make a plot base on the type
            if plot_type=='line':
                fig.line(x=flanker_test_dic[key][x_axis], y=flanker_test_dic[key][y_axis],
                    legend_label= key, line_width=0.5, color=colors[counter])
                
            elif plot_type=='scatter':
                fig.scatter( x=flanker_test_dic[key][x_axis], y=flanker_test_dic[key][y_axis],
                        legend_label= key, marker='circle', fill_color=colors[counter], line_color='white', size=5)
                
            elif plot_type=='vbar':
                df = pd.DataFrame(flanker_test_dic[key])
                dff = pd.DataFrame(flanker_test_dic[key].groupby([x_axis]).count())            
                fig.vbar(x=dff.index, width=0.5, bottom=0, top=dff[y_axis], 
                        color=colors[counter], legend_label= key)   

    return fig

In [316]:
#making an interactive plot
columns_name = ['index', 'homology', 'accuracy', 'reaction_time']
plot_name =  ['line', 'scatter', 'vbar']
inter_plot = pn.interact(render_plot, x_axis=columns_name, y_axis=columns_name, plot_type=plot_name)

In [318]:
#making a dashboard of interactive plot
dashboard = pn.template.BootstrapTemplate(title='Flanker Test',sidebar_width=200)
dashboard.sidebar.append(inter_plot[0])
dashboard.main.append(inter_plot[1])
dashboard.show()

Launching server at http://localhost:63094
